In [7]:
import cProfile
import os
import pstats
import sys
from pstats import SortKey

import numpy as np
import pandas as pd
from river.decomposition import OnlineDMDwC
from river.preprocessing import Hankelizer
from river.utils import Rolling
from tqdm import tqdm

sys.path.append("../")

from functions.chdsubid import (
    SubIDChangeDetector,
    get_default_rank,
)
from functions.datasets import load_cats
from functions.metrics import chp_score
from functions.plot import plot_chd
from functions.preprocessing import hankel

Rolling.learn_one = Rolling.update  # type: ignore

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Read data
df = load_cats(resample_s=60)
df.index = pd.to_datetime(df.index)
U = df[["aimp", "amud", "adbr", "adfl"]]
X = df[
    [
        "bed1",
        "bed2",
        "bfo1",
        "bfo2",
        "bso1",
        "bso2",
        "bso3",
        "ced1",
        "cfo1",
        "cso1",
    ]
]
Y_true = df.pop("y")
Y_cat = df.pop("category")

In [84]:
# What kind of structural changes are we looking for? Steps every 1000 samples
window_size: int = 1000
# Base size
ref_size = window_size
test_size = 300
# If window_size is not very large, then take half
hn = 2
# Optimal low-rank representation of signal with unknown noise variance
p = min(len(X.columns), get_default_rank(hankel(X[:window_size], hn)))
print(f"Optimal rank for states is: {p}")
hu = window_size // 2
q = min(len(U.columns), get_default_rank(hankel(U[:window_size], hu)))
print(f"Optimal rank for inputs is: {q}")

Optimal rank for states is: 8
Optimal rank for inputs is: 4


In [ ]:
# [donotremove]
# TODO: enable hankelization of us on the fly
U_ = pd.DataFrame(hankel(U, hn=hu))

# Initialize Hankelizer
hankelizer = Hankelizer(hn)

# Initialize Transformer
init_size = window_size
odmd = Rolling(
    OnlineDMDwC(
        p=p,
        q=q,
        initialize=init_size,
        w=1.0,
        exponential_weighting=False,
        eig_rtol=1e-1,
    ),
    init_size + 1,
)

# Initialize Change-Point Detector
subid_dmd = SubIDChangeDetector(
    odmd,
    ref_size=ref_size,
    test_size=test_size,
    grace_period=init_size + test_size + 1,
)

# Build pipeline
pipeline_dmd = hankelizer | subid_dmd

# Prepare arrays for storing results
scores_dmd = np.zeros(X.shape[0], dtype=float)
# scores_svd = np.zeros(X.shape[0], dtype=float)
scores_dmd_diff = np.zeros(X.shape[0], dtype=complex)
dist = np.zeros((X.shape[0], 2), dtype=complex)
modes = np.empty((X.shape[0], hn * X.shape[1], p), dtype=complex)

# Run pipeline
for i, (x, u) in tqdm(
    enumerate(
        zip(
            X.to_dict(orient="records"),
            U_.to_dict(orient="records"),
        )
    ),
    total=len(X),
    mininterval=1.0,
    desc="Snapshots processed",
):
    scores_dmd[i] = pipeline_dmd.score_one(x)
    dist[i, :] = subid_dmd.distances
    scores_dmd_diff[i] = dist[i, 1] - dist[i, 0]

    pipeline_dmd.learn_one(x, **{"u": u})

# Plot results
X_ = df.values
Y_ = Y_true.values
fig, axs = plot_chd(
    [X_, scores_dmd.real, scores_dmd_diff.real],
    np.where(Y_ == 1)[0],
    labels=["X", "DMD", "DMD_diff", "SVD"],
    ids_in_start=[30000],
    ids_in_end=[40000],
    grace_period=test_size,
)
fig_name = (
    f"results/.cats/cats-chd_p{p}_q{q}-l{init_size}_b{ref_size}_t{test_size}"
    f"{f'roll_{odmd.window_size}' if isinstance(odmd, Rolling) else 'noroll'}-"
    f"dmd_w{odmd.w}-hx{hn}-hu{hu}-imag.pdf"
)
fig.set_size_inches(18, 10)  # Set the size of the figure
os.makedirs(os.path.dirname(fig_name), exist_ok=True)
fig.savefig(fig_name)

In [17]:
pred_shift = 1
y_true = pd.Series(Y_)
y_true.index = pd.to_datetime(y_true.index, unit="60s")
y_true_cp = y_true.diff().abs()
y_true_cp.iloc[:1] = 0.0

y_pred = pd.Series(scores_dmd > 0.075)
y_pred.index = pd.to_datetime(y_pred.index, unit="60s")

y_cp = y_pred.diff().abs()
y_cp.iloc[:1] = 0.0

In [18]:
binary = chp_score(y_true, y_pred, metric="binary")

False Alarm Rate 30.49 %
Missing Alarm Rate 59.33 %
F1 metric 0.09


In [19]:
# average detection delay metric calculation
add = chp_score(
    y_true,
    y_cp,
    metric="average_time",
    window_width=f"{test_size}s",
    anomaly_window_destination="righter",
    portion=1,
)

Amount of true anomalies 3174
A number of missed CPs = 3054
A number of FPs = 198
Average time 0 days 00:01:10.725000


In [20]:
# nab metric calculation
nab = chp_score(
    y_true,
    y_pred,
    metric="nab",
    window_width=f"{test_size}s",
    anomaly_window_destination="righter",
)

Standard  -  30.45
LowFP  -  15.87
LowFN  -  35.42


# Time Profiling to Reveal the Main Bottlenecks

In [86]:
import datetime


def simulate():
    # Initialize Hankelizer
    U_ = pd.DataFrame(hankel(U, hn=hu))

    # Initialize Hankelizer
    hankelizer = Hankelizer(hn)

    # Initialize Transformer
    init_size = window_size
    odmd = Rolling(
        OnlineDMDwC(
            p=p,
            q=q,
            initialize=init_size,
            w=1.0,
            exponential_weighting=False,
            eig_rtol=1e-1,
        ),
        init_size + 1,
    )

    # Initialize Change-Point Detector
    subid_dmd = SubIDChangeDetector(
        odmd,
        ref_size=ref_size,
        test_size=test_size,
        grace_period=init_size + test_size + 1,
    )

    # Build pipeline
    pipeline_dmd = hankelizer | subid_dmd

    # Prepare arrays for storing results
    scores_dmd = np.zeros(X.shape[0], dtype=float)
    # Run pipeline
    for i, (x, u) in tqdm(
        enumerate(
            zip(
                X.to_dict(orient="records"),
                U_.to_dict(orient="records"),
            )
        ),
        total=len(X),
        mininterval=10.0,
        desc="Snapshots processed",
    ):
        scores_dmd[i] = pipeline_dmd.score_one(x)

        pipeline_dmd.learn_one(x, **{"u": u})


cProfile.run(
    "simulate()",
    f".stats_{datetime.datetime.now(datetime.UTC).strftime("%Y-%m-%d_%H%M%S")}",
)

Snapshots processed: 100%|██████████| 83274/83274 [30:26<00:00, 45.60it/s]


In [87]:
perf_stats = pstats.Stats(".stats_pd_2024-05-17_225443")
perf_stats.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats()

Sat May 18 08:25:42 2024    .stats_pd_2024-05-17_225443

         602901271 function calls (588544191 primitive calls) in 1859.799 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  369/207    4.293    0.012 1875.361    9.060 threading.py:323(wait)
      2/1    0.000    0.000 1855.906 1855.906 {built-in method builtins.exec}
        1    0.059    0.059 1855.906 1855.906 <string>:1(<module>)
        1    9.610    9.610 1851.511 1851.511 2430896644.py:4(simulate)
    83274    1.009    0.000 1120.741    0.013 pipeline.py:418(learn_one)
    83274    0.182    0.000 1116.805    0.013 chdsubid.py:245(learn_one)
    83274    0.389    0.000 1116.606    0.013 chdsubid.py:309(update)
    82974    0.994    0.000 1115.934    0.013 rolling.py:80(update)
   163945  131.676    0.001 1029.830    0.006 odmd.py:286(_truncate_w_svd)
    82974   22.103    0.000  846.359    0.010 odmd.py:970(update)
    81973    3.465    0.000  804.141    0.010

In [90]:
perf_stats = pstats.Stats(".stats_new")
perf_stats.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats()

Tue Apr  9 08:10:09 2024    .stats_new

         311968672 function calls (305153401 primitive calls) in 713.911 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000  713.911  713.911 {built-in method builtins.exec}
        1    0.021    0.021  713.911  713.911 <string>:1(<module>)
        1    0.115    0.115  713.891  713.891 1745487788.py:4(simulate)
     5000    0.105    0.000  712.637    0.143 pipeline.py:418(learn_one)
     5000    0.534    0.000  709.902    0.142 chdsubid.py:137(learn_one)
     5000    2.063    0.000  709.369    0.142 chdsubid.py:93(update)
    27000    0.388    0.000  551.492    0.020 frame.py:683(__init__)
     4500    3.011    0.001  452.800    0.101 construction.py:506(nested_data_to_arrays)
     4500    0.129    0.000  449.668    0.100 construction.py:793(to_arrays)
     4500  286.930    0.064  333.052    0.074 construction.py:891(_list_of_dict_to_arrays)
     4500    2.1